### Python使用Faiss实现向量近邻搜索

Embedding的近邻搜索是当前图推荐系统非常重要的一种召回方式，通过item2vec、矩阵分解、双塔DNN等方式都能够产出训练好的user embedding、item embedding，对于embedding的使用非常的灵活：

* 输入user embedding，近邻搜索item embedding，可以给user推荐感兴趣的items
* 输入user embedding，近邻搜搜user embedding，可以给user推荐感兴趣的user
* 输入item embedding，近邻搜索item embedding，可以给item推荐相关的items

然而有一个工程问题，一旦user embedding、item embedding数据量达到一定的程度，对他们的近邻搜索将会变得非常慢，如果离线阶段提前搜索好在高速缓存比如redis存储好结果当然没问题，但是这种方式很不实时，如果能在线阶段上线几十MS的搜索当然效果最好。

Faiss是Facebook AI团队开源的针对聚类和相似性搜索库，为稠密向量提供高效相似度搜索和聚类，支持十亿级别向量的搜索，是目前最为成熟的近似近邻搜索库。

安装命令：   
```
conda install -c pytorch faiss-cpu 
```

演示步骤：
1. 读取训练好的Embedding数据
2. 构建faiss索引，将待搜索的Embedding添加进去
3. 取得目标Embedding，实现搜索得到ID列表
4. 根据ID获取电影标题，返回结果

faiss使用经验：
1. 为了支持自己的ID，可以用faiss.IndexIDMap包裹faiss.IndexFlatL2即可
2. embedding数据都需要转换成np.float32，包括索引中的embedding以及待搜索的embedding
3. ids需要转换成int64类型

### 1. 准备数据

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./datas/movielens_sparkals_item_embedding.csv")
df.head()

,id,features
0,10,"[0.25866490602493286, 0.3560594320297241, 0.15..."
1,20,"[0.12449632585048676, -0.29282501339912415, -0..."
2,30,"[0.9557555317878723, 0.6764761805534363, 0.114..."
3,40,"[0.3184879720211029, 0.6365472078323364, 0.596..."
4,50,"[0.45523127913475037, 0.34402626752853394, -0...."


#### 构建ids

In [3]:
ids = df["id"].values.astype(np.int64)
type(ids), ids.shape

(numpy.ndarray, (3706,))

In [4]:
ids.dtype

dtype('int64')

In [5]:
ids_size = ids.shape[0]
ids_size

3706

#### 构建datas

In [6]:
import json
import numpy as np

In [7]:
datas = []

In [8]:
for x in df["features"]:
    datas.append(json.loads(x))

In [9]:
datas = np.array(datas).astype(np.float32)

In [10]:
datas.dtype

dtype('float32')

In [11]:
datas.shape

(3706, 10)

In [12]:
datas[0]

array([ 0.2586649 ,  0.35605943,  0.15589039, -0.7067125 , -0.07414215,
       -0.62500805, -0.0573845 ,  0.4533663 ,  0.26074877, -0.60799956],
      dtype=float32)

In [13]:
# 维度
dimension = datas.shape[1]
dimension

10

### 2. 建立索引

In [14]:
import faiss

In [15]:
index = faiss.IndexFlatL2(dimension)

In [16]:
index2 = faiss.IndexIDMap(index)

In [17]:
ids.dtype

dtype('int64')

In [18]:
index2.add_with_ids(datas, ids)

In [19]:
index.ntotal

3706

### 3. 搜索近邻ID列表

In [20]:
df_user = pd.read_csv("./datas/movielens_sparkals_user_embedding.csv")
df_user.head()

,id,features
0,10,"[0.5974288582801819, 0.17486965656280518, 0.04..."
1,20,"[1.3099910020828247, 0.5037978291511536, 0.260..."
2,30,"[-1.1886241436004639, -0.13511677086353302, 0...."
3,40,"[1.0809299945831299, 1.0048035383224487, 0.986..."
4,50,"[0.42388680577278137, 0.5294889807701111, -0.6..."


In [21]:
user_embedding = np.array(json.loads(df_user[df_user["id"] == 10]["features"].iloc[0]))
user_embedding = np.expand_dims(user_embedding, axis=0).astype(np.float32)
user_embedding

array([[ 0.59742886,  0.17486966,  0.04345559, -1.3193961 ,  0.5313592 ,
        -0.6052168 , -0.19088413,  1.5307966 ,  0.09310367, -2.7573566 ]],
      dtype=float32)

In [22]:
user_embedding.shape

(1, 10)

In [23]:
user_embedding.dtype

dtype('float32')

In [24]:
topk = 30

In [26]:
D, I = index2.search(user_embedding, topk)     # actual search

In [27]:
I.shape

(1, 30)

In [28]:
I

array([[ 439, 3147,  985, 1290, 3408, 2762, 2671,   62, 1286, 2501, 1246,
         440, 1636, 1871, 3044, 2572, 3255,  708,  597,   11, 3448, 2797,
        2396, 1961, 3512, 2125, 3479,   49,  539, 2739]])

### 4. 根据电影ID取出电影信息

In [29]:
target_ids = pd.Series(I[0], name="MovieID")
target_ids.head()

0     439
1    3147
2     985
3    1290
4    3408
Name: MovieID, dtype: int64

In [30]:
df_movie = pd.read_csv("./datas/ml-1m/movies.dat",
                     sep="::", header=None, engine="python",
                     names = "MovieID::Title::Genres".split("::"))
df_movie.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [31]:
df_result = pd.merge(target_ids, df_movie)
df_result.head()

,MovieID,Title,Genres
0,439,Dangerous Game (1993),Drama
1,3147,"Green Mile, The (1999)",Drama|Thriller
2,985,Small Wonders (1996),Documentary
3,1290,Some Kind of Wonderful (1987),Drama|Romance
4,3408,Erin Brockovich (2000),Drama
